In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import sys
import timeit
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import asarray 
from datasets import load_dataset 
from sklearn import model_selection
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from vit_keras import vit
import warnings
from PIL import Image
import csv
import random

In [ ]:
#######################
# De todas las configuraciones realizadas en el trabajo anterior, usamos CF2 la cuál es la de salida binaria benigno/maligno
CF = 'CF2'
#######################

#Especifique los directorios necesarios
#######################
path_or = '.\\Datos\\'
path_img = path_or + f'{CF}\\imagenes\\'
input_path = path_or + f'{CF}\\salidas\\'
new_images = path_or + f'{CF}\\new_images\\'
#######################

In [ ]:
#Elige si convertir en variable dataset todos los nombres y clases de las imagenes
#######################
leer_imagenes_flag = True
#######################

#Elige si guardar el resultado final en archivos .npy (y_train.npy, x_test.npy...)
#######################
redimensionar_datos_224_flag = True
#######################

# Dimensiones de las imágenes para redes preentrenadas
WIDTH_224 = 224
HEIGHT_224 = 224

# Dimensiones de las imágenes para la representación visual
WIDTH_VIZ = 600
HEIGHT_VIZ = 600

# Agrupamiento Datasets (HIBA, HAM, MSKCC)

In [ ]:
dataset_hiba = pd.read_csv('./ISIC_HIBASkinLesionsData/final_metadata_HIBA.csv')
dataset_sevilla = pd.read_csv('./Sevilla_Data/final_metadata_Sevilla.csv')
dataset_mskcc = pd.read_csv('./ISIC_MSKCC/final_metadata_MSKCC.csv')

In [ ]:
dataset_hiba

In [ ]:
dataset_sevilla

In [ ]:
dataset_mskcc

In [ ]:
dataset_factores_externos = pd.concat([dataset_hiba, dataset_sevilla, dataset_mskcc], ignore_index=True)

# Procesamiento Factores externos

In [ ]:
dataset_factores_externos

In [ ]:
# Eliminamos las columnas que no serán utilizadas
dataset_factores_externos = dataset_factores_externos.drop(columns=["anatom_site_general","sex","age","diagnosis"])

In [ ]:
# Normalizamos la edad 
scaler = MinMaxScaler()

# Lista de columnas que quieres transformar
columns_to_transform = [
    "max_dew_point", "min_dew_point", "mean_dew_point", "median_dew_point", "std_dew_point",
    "max_wind_speed", "min_wind_speed", "mean_wind_speed", "median_wind_speed", "std_wind_speed",
    "max_wind_direction", "min_wind_direction", "mean_wind_direction", "median_wind_direction", "std_wind_direction",
    "max_visibility", "min_visibility", "mean_visibility", "median_visibility", "std_visibility",
    "max_temp", "min_tmp", "mean_temp", "median_temp", "std_temp", 'ALLSKY_KT_min', 'ALLSKY_KT_max', 'ALLSKY_KT_mean', 'ALLSKY_KT_median', 
    'ALLSKY_KT_std', 'CLOUD_AMT_min', 'CLOUD_AMT_max', 'CLOUD_AMT_mean', 'CLOUD_AMT_median', 'CLOUD_AMT_std', 
    'TOA_SW_DWN_min', 'TOA_SW_DWN_max', 'TOA_SW_DWN_mean', 'TOA_SW_DWN_median', 'TOA_SW_DWN_std', 
    'ALLSKY_SFC_UVA_min', 'ALLSKY_SFC_UVA_max', 'ALLSKY_SFC_UVA_mean', 'ALLSKY_SFC_UVA_median', 
    'ALLSKY_SFC_UVA_std', 'ALLSKY_SFC_UVB_min', 'ALLSKY_SFC_UVB_max', 'ALLSKY_SFC_UVB_mean', 'ALLSKY_SFC_UVB_median', 
    'ALLSKY_SFC_UVB_std', 'ALLSKY_SRF_ALB_min', 'ALLSKY_SRF_ALB_max', 'ALLSKY_SRF_ALB_mean', 'ALLSKY_SRF_ALB_median', 
    'ALLSKY_SRF_ALB_std', 'ALLSKY_SFC_SW_DNI_min', 'ALLSKY_SFC_SW_DNI_max', 'ALLSKY_SFC_SW_DNI_mean', 'ALLSKY_SFC_SW_DNI_median', 
    'ALLSKY_SFC_SW_DNI_std', 'ALLSKY_SFC_SW_DWN_min', 'ALLSKY_SFC_SW_DWN_max', 'ALLSKY_SFC_SW_DWN_mean', 'ALLSKY_SFC_SW_DWN_median', 
    'ALLSKY_SFC_SW_DWN_std', 'ALLSKY_SFC_PAR_TOT_min', 'ALLSKY_SFC_PAR_TOT_max', 'ALLSKY_SFC_PAR_TOT_mean', 'ALLSKY_SFC_PAR_TOT_median', 
    'ALLSKY_SFC_PAR_TOT_std', 'ALLSKY_SFC_SW_DIFF_min', 'ALLSKY_SFC_SW_DIFF_max', 'ALLSKY_SFC_SW_DIFF_mean', 'ALLSKY_SFC_SW_DIFF_median', 
    'ALLSKY_SFC_SW_DIFF_std']


for column in columns_to_transform:
    dataset_factores_externos[column] = scaler.fit_transform(np.array(dataset_factores_externos[column]).reshape(-1, 1))

dataset_factores_externos.reset_index(drop=True, inplace=True)

In [ ]:
dataset_factores_externos

In [ ]:
dataset_factores_externos_corr = dataset_factores_externos.drop(columns=["id"])
corr_matrix = dataset_factores_externos_corr.corr().abs()

columna_objetivo = 'benign_malignant'  
correlaciones_con_columna_objetivo = corr_matrix[columna_objetivo]

In [ ]:
# Filtramos por correlaciones significativas por encima de un 0.2
correlaciones_significativas = correlaciones_con_columna_objetivo[(correlaciones_con_columna_objetivo > 0.2) | (correlaciones_con_columna_objetivo < -0.2)]
correlaciones_significativas_ordenadas = correlaciones_significativas.sort_values(ascending=False)

print(correlaciones_significativas_ordenadas.head(10))

In [ ]:
# Nos quedamos con las columnas que utilizaremos en nuestro modelo
columnas_seleccionadas = ['id','ALLSKY_SFC_PAR_TOT_min', 'std_temp', 
                          'ALLSKY_SFC_SW_DWN_min', 'benign_malignant'  ]

dataset_factores_externos = dataset_factores_externos[columnas_seleccionadas].copy()
dataset_factores_externos

# Procesamiento imágenes

In [ ]:
# Diccionario con las classes CF2
classes = {0: ('ben', 'BENIGNO'),
           1: ('mal', 'MALIGNO')}

cm_plot_classes = ['ben', 'mal']

In [ ]:
data=[]
with open(path_or + 'metadata_CF2.csv', 'w', newline='') as writeFile:
    writer = csv.writer(writeFile, delimiter=';')
    writer.writerow(['image', 'label'])
    for filename in os.listdir('./Datos/CF2/imagenes/BENIGNO/'):
        writer.writerow(['BENIGNO/' + filename, '0'])
    for filename in os.listdir('./Datos/CF2/imagenes/MALIGNO/'):
        writer.writerow(['MALIGNO/' + filename, '1'])
    writeFile.close()

In [ ]:
metadata = pd.read_csv(path_or + f'metadata_{CF}.csv', delimiter=';')
metadata

In [ ]:
# Generamos una nueva columna 'id' con el nombre de cada una de las imágenes para poder tener un identificador con el que ordenar ambos datasets
metadata['id'] = metadata['image'].str.split('/').str[1].str.replace('.jpg', '').str.replace('.JPG', '')

In [ ]:
# Ordenamos ambos datasets por el ID
metadata = metadata.sort_values(by='id')
dataset_factores_externos = dataset_factores_externos.sort_values(by='id')

In [ ]:
metadata

In [ ]:
dataset_factores_externos

In [ ]:
metadata = metadata.drop(columns=["id"])
dataset_factores_externos = dataset_factores_externos.drop(columns=["id"])

In [ ]:
#Lo pasamos a float16 para ajustar memoria y optimizar rendimiento
dataset_factores_externos = dataset_factores_externos.astype(np.float16)

In [ ]:
# Definimos X e Y para entrenamiento de los factores externos
aux = dataset_factores_externos.drop(columns=["benign_malignant"])
y_ext = dataset_factores_externos['benign_malignant']
x_ext = aux.values # Generamos el conjunto X y lo pasamos a numpy array

In [ ]:
# Distribución de frecuencia de imágenes de cada clase
plt.figure(figsize = (12, 8))
sns.set_theme(style = 'darkgrid')
sns.countplot(data = metadata, x = 'label', order = metadata['label'].value_counts().index, palette = 'mako')
plt.xlabel('Clase', size = 15)
plt.ylabel('Cantidad', size = 15)
plt.title(f'Distribución de frecuencia de imágenes de cada clase en el {CF}', size = 20)

In [ ]:
# Guardamos el nuevo dataset en formato csv
dataset_factores_externos.to_csv(path_or + 'external_factors_metadata_CF2.csv', index=False)

In [ ]:
metadata['label'].value_counts()

In [ ]:
metadata

In [ ]:
if leer_imagenes_flag == True:
    dataset = load_dataset('csv', data_files=[path_or+f'metadata_{CF}.csv'], delimiter=';')
    y = dataset['train']['label']
    x = dataset['train']['image']

In [ ]:
def modificar_resolucion_imagenes(x, conjunto_nuevo, anchura, altura):
    resolucion = (anchura, altura)
    for imagen in x:
        imagen_pil = Image.open(path_img+imagen)
        imagen2 = imagen_pil.convert("RGB")
        imagen_redimensionada = imagen2.resize(resolucion)
        conjunto_nuevo.append(imagen_redimensionada)

In [ ]:
if redimensionar_datos_224_flag == True:
    x2 = []
    modificar_resolucion_imagenes(x, x2, WIDTH_224, HEIGHT_224)

    x2_nor = []
    for imagen in x2:
        imagen2 = imagen.convert("RGB")
        # Convertir la imagen a un arreglo NumPy
        img_array = np.array(imagen2)
        x2_nor.append(img_array)
    x2_nor = (x2_nor - np.mean(x2_nor)) / np.std(x2_nor)

    x2_nor = np.array(x2_nor)
    x2_nor = x2_nor.reshape(-1, WIDTH_224, HEIGHT_224, 3)
    y = np.array(y)

    # Imagenes
    x_train, x_test, y_train, y_test = model_selection.train_test_split(x2_nor, y, test_size=0.2, random_state=1)
    x_test, x_val, y_test, y_val = model_selection.train_test_split(x_test, y_test, test_size=0.5, random_state=1)

    # Factores externos
    x_train_ext, x_test_ext, y_train_ext, y_test_ext = model_selection.train_test_split(x_ext, y_ext, test_size=0.2, random_state=1)
    x_test_ext, x_val_ext, y_test_ext, y_val_ext = model_selection.train_test_split(x_test_ext, y_test_ext, test_size=0.5, random_state=1)

    # Verificar las dimensiones
    print("Dimensiones de x_train:", x_train.shape)
    print("Dimensiones de x_val:", x_val.shape)
    print("Dimensiones de x_test:", x_test.shape)

    # Verificar las dimensiones
    print("Dimensiones de x_train_ext:", x_train_ext.shape)
    print("Dimensiones de x_val_ext:", x_val_ext.shape)
    print("Dimensiones de x_test_ext:", x_test_ext.shape)

    # Conversión a ndarrays de números reales
    x_train = np.asarray(x_train).astype(np.float16)
    np.save(input_path + 'x_train', x_train)

    x_test = np.asarray(x_test).astype(np.float16)
    np.save(input_path + 'x_test', x_test)
    
    x_val = np.asarray(x_val).astype(np.float16)
    np.save(input_path + 'x_val', x_val)

    # Conversión a ndarrays de números reales
    x_train_ext = np.asarray(x_train_ext).astype(np.float16)
    np.save(input_path + 'x_train_ext', x_train_ext)

    x_test_ext = np.asarray(x_test_ext).astype(np.float16)
    np.save(input_path + 'x_test_ext', x_test_ext)
    
    x_val_ext = np.asarray(x_val_ext).astype(np.float16)
    np.save(input_path + 'x_val_ext', x_val_ext)
    
    y_train = np.asarray(y_train).astype(bool)
    y_test = np.asarray(y_test).astype(bool)
    y_val = np.asarray(y_val).astype(bool)

    y_train_ext = np.asarray(y_train_ext).astype(bool)
    y_test_ext = np.asarray(y_test_ext).astype(bool)
    y_val_ext = np.asarray(y_val_ext).astype(bool)

    # Imagenes
    np.save(input_path + 'y_train', y_train)
    np.save(input_path + 'y_test', y_test)
    np.save(input_path + 'y_val', y_val)

    # Factores externos
    np.save(input_path + 'y_train_ext', y_train_ext)
    np.save(input_path + 'y_test_ext', y_test_ext)
    np.save(input_path + 'y_val_ext', y_val_ext)